# Objectif du notebook :
Indexer les documents pdfs pas encore indexés dans une collection existante.

In [ ]:
import os
import requests
from openai import OpenAI
import glob
from typing import Dict, Optional
import json
import time
from pathlib import Path

# Paramètres d'entrée

In [ ]:
id_collection_existante = int(input("Insérez le numéro de la collection ID à laquelle vous souhaitez ajouter de "))
nom = input("Entrez le nom complet du fichier pdf à ajouter : ")
url = input("Entrez l'URL du fichier à ajouter : ")

In [ ]:
base_url = "https://albert.api.etalab.gouv.fr/v1"
api_cle = os.getenv("ALBERT_API_KEY")
client = OpenAI(base_url=base_url, api_key=api_cle)
session = requests.session()
session.headers = {"Authorization": f"Bearer {api_cle}"}

# Fonctions utilitaires

In [ ]:
def obtient_documents(session, base_url, collection_id, limit=10, offset=0):
    params = {
        "collection": collection_id,
        "limit": limit,
        "offset": offset,
    }
    response = session.get(f"{base_url}/documents", params=params)
    response.raise_for_status()
    return response.json()


def obtient_noms_uniques(session, base_url, collection_id, limit=100, offset=0):
    docs = obtient_documents(session, base_url, collection_id, limit, offset)
    names = {doc["name"] for doc in docs.get("data", []) if "name" in doc}
    return sorted(names)

### Définition des Modèles LLMs utilisés

In [ ]:
model_langue, model_embedding = None, None

for model in client.models.list().data:
    if model.type == "text-generation" and model_langue is None:
        model_langue = model.id
    if model.type == "text-embeddings-inference" and model_embedding is None:
        model_embedding = model.id
        
print(f"language model: {model_langue}\nembeddings model: {model_embedding}")

# Ajouter des pdfs non indexés

In [ ]:
fichiers = glob.glob("/home/pleroy/Downloads/ANSSI/*.pdf")

#### Pour ajouter un nouveau document il faut ajouter le nom du pdf ainsi que l'url du pdf

In [ ]:
# Dictionnaire nom -> URL (faites correspondre à vos fichiers locaux)
URLS_PAR_NOM: Dict[str, str] = {
    "anssi-guide-authentification_multifacteur_et_mots_de_passe.pdf": "https://cyber.gouv.fr/sites/default/files/2021/10/anssi-guide-authentification_multifacteur_et_mots_de_passe.pdf",
    "anssi-guide-gestion_crise_cyber.pdf": "https://cyber.gouv.fr/sites/default/files/2021/12/anssi-guide-gestion_crise_cyber.pdf",
    "guide_hygiene_informatique_anssi.pdf": "https://cyber.gouv.fr/sites/default/files/2017/01/guide_hygiene_informatique_anssi.pdf",
    "guide_nomadisme_anssi_pa_054_v2.pdf": "https://cyber.gouv.fr/sites/default/files/document/guide_nomadisme_anssi_pa_054_v2.pdf",
    "anssi-guide-admin_securisee_si_v3-0.pdf": "https://cyber.gouv.fr/sites/default/files/2018/04/anssi-guide-admin_securisee_si_v3-0.pdf",
    "anssi-guide-passerelle_internet_securisee-v3.pdf": "https://cyber.gouv.fr/sites/default/files/2020/06/anssi-guide-passerelle_internet_securisee-v3.pdf",
    "anssi-fondamentaux-sauvegarde_systemes_dinformation_v1-0.pdf": "https://cyber.gouv.fr/sites/default/files/document/anssi-fondamentaux-sauvegarde_systemes_dinformation_v1-0.pdf",
    "guide_protection_des_systemes_essentiels.pdf": "https://cyber.gouv.fr/sites/default/files/2020/12/guide_protection_des_systemes_essentiels.pdf",
    "guide-homologation-securite-web-04-2025.pdf": "https://cyber.gouv.fr/sites/default/files/document/guide-homologation-securite-web-04-2025.pdf",
    "anssi-guide-recommandations_mise_en_oeuvre_site_web_maitriser_standards_securite_cote_navigateur-v2.0.pdf": "https://cyber.gouv.fr/sites/default/files/2013/05/anssi-guide-recommandations_mise_en_oeuvre_site_web_maitriser_standards_securite_cote_navigateur-v2.0.pdf",
    "secnumcloud-referentiel-exigences-v3.2.pdf": "https://cyber.gouv.fr/sites/default/files/document/secnumcloud-referentiel-exigences-v3.2.pdf",
    "LAB_Homologation_Simplifiee.pdf": "https://monservicesecurise-ressources.cellar-c2.services.clever-cloud.com/LAB_Homologation_Simplifiee.pdf",
}

In [ ]:

URLS_PAR_NOM[nom] = url

print("Ajouté :", nom, "->", url)
print("Dictionnaire mis à jour :")
for k, v in URLS_PAR_NOM.items():
    print(f"{k} -> {v}")

In [ ]:


def ajouter_fichier(
    chemin_fichier: str,
    id_collection: str,
    base_url: str,
    url_source: Optional[str] = None,
) -> requests.Response:
    """
    Téléverse un PDF dans Albert et ajoute l'URL d'origine en métadonnée.

    :param chemin_fichier: chemin local du PDF.
    :param id_collection: identifiant de la collection cible.
    :param url_source: URL publique du document.
    :return: réponse HTTP de l’API.
    """
    nom = Path(chemin_fichier).name
    with open(chemin_fichier, "rb") as flux:
        fichiers = {"file": (nom, flux, "application/pdf")}
        donnees = {
            "collection": str(id_collection),
            "metadata": json.dumps({"source_url": url_source}),
        }
        # autres options possibles :
        # "paginate_output": "false",
        # "force_ocr": "false",
        # "output_format": "markdown",
        # "chunk_size": "2048",
        reponse = session.post(f"{base_url}/documents", data=donnees, files=fichiers)
    return reponse

    
def url_pour(chemin_fichier: str) -> Optional[str]:
    """Retourne l’URL mappée à partir du nom de fichier, sinon None."""
    nom = Path(chemin_fichier).name
    return URLS_PAR_NOM.get(nom)

def est_deja_indexe(collection_id: str, nom_fichier: str) -> bool:
    """
    Vérifie si un document avec ce nom existe déjà dans la collection.
    """
    documents = liste_dcouments_dans_collection(collection_id)["data"]
    noms = {doc["name"] for doc in documents}
    return nom_fichier in noms


In [ ]:
def obtient_collections(id_collection):
    response = session.get(f"{base_url}/collections/{id_collection}")
    response = response.json()
    return response

def liste_dcouments_dans_collection(id_collection):
    params = {
        "collection": id_collection,
        "limit": 100,
        "offset": 0,
    }
    response = session.get(f"{base_url}/documents", params=params)
    return response.json()


In [ ]:
nombre_documents_indexes = obtient_collections(id_collection_existante)["documents"]

for chemin in fichiers:
    nom_fichier = Path(chemin).name
    # Vérification préalable
    if est_deja_indexe(id_collection_existante, nom_fichier):
        print(f"[SKIP] Déjà indexé : {nom_fichier}")
        continue

    url = url_pour(chemin)
    succes = False
    tentative = 0

    while tentative < 3 and not succes:
        tentative += 1
        print(f"Tentative {tentative} pour {nom_fichier} ...")

        r = ajouter_fichier(
            chemin,
            id_collection_existante,
            base_url=base_url,
            url_source=url,
        )

        nombre_documents_indexes_actuellement = obtient_collections(id_collection_existante)[
            "documents"
        ]

        if nombre_documents_indexes_actuellement - nombre_documents_indexes == 1:
            print(f"Le document a été indexé : {chemin}")
            succes = True
            nombre_documents_indexes = nombre_documents_indexes_actuellement
        else:
            print(f"Le document n'a pas été indexé : {chemin}")
            if tentative < 3:
                print("Nouvel essai dans 5 secondes...")
                time.sleep(5)
            else:
                print("Échec après 3 tentatives.")

        print(r.status_code)
        print("*" * 10)
        try:
            print(r.json())
        except Exception:
            print(f"ERROR : {r.text}")